# **Chapter 4: Arrays and Dynamic Arrays**

> *"The array is the fundamental data structure. Understanding arrays is understanding how computers think about memory."*

---

## **4.1 Introduction**

Arrays are the most fundamental and widely-used data structure in computer science. They provide **constant-time random access** to elements stored in **contiguous memory locations**, making them the building block for virtually all other data structures.

This chapter explores arrays from first principles: their memory layout, performance characteristics, dynamic resizing strategies, and practical applications in algorithms and system design.

---

## **4.2 Static Arrays: Memory Layout and Cache Locality**

### **4.2.1 What is a Static Array?**

A **static array** is a fixed-size collection of elements of the same type, stored in contiguous memory locations. The size is determined at compile time and cannot be changed during runtime.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    STATIC ARRAY PROPERTIES                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Size:           Fixed at compile time                              │
│  Memory:         Contiguous allocation                              │
│  Element Access: O(1) by index                                      │
│  Element Type:   Homogeneous (same type)                            │
│  Declaration:    int arr[5];  // C/C++                              │
│                  int[] arr = new int[5];  // Java                   │
│                                                                      │
│  Memory Layout:                                                      │
│                                                                      │
│  Index:       [0]    [1]    [2]    [3]    [4]                      │
│               ┌──────┬──────┬──────┬──────┬──────┐                 │
│  Elements:    │  10  │  20  │  30  │  40  │  50  │                 │
│               └──────┴──────┴──────┴──────┴──────┘                 │
│  Address:     1000   1004   1008   1012   1016                     │
│               (assuming 4-byte integers)                            │
│                                                                      │
│  Address Formula: address(arr[i]) = base_address + i × element_size │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

### **4.2.2 Memory Layout and Address Calculation**

```python
def memory_layout_demo():
    """
    Demonstrate array memory layout and address calculation.
    """
    import ctypes
    
    print("Array Memory Layout")
    print("=" * 70)
    
    print("""
    Physical Memory Layout:
    ─────────────────────────────────────────────────────────────────────
    
    Consider the array: int arr[6] = {10, 20, 30, 40, 50, 60};
    
    Assuming:
      • Base address: 0x1000
      • Element size (int): 4 bytes
      • Total memory: 6 × 4 = 24 bytes
    
    Memory Map:
    
    Address    │  Byte 0  │  Byte 1  │  Byte 2  │  Byte 3  │  Value
    ───────────┼──────────┼──────────┼──────────┼──────────┼─────────
    0x1000     │    10    │    00    │    00    │    00    │   arr[0]
    0x1004     │    20    │    00    │    00    │    00    │   arr[1]
    0x1008     │    30    │    00    │    00    │    00    │   arr[2]
    0x100C     │    40    │    00    │    00    │    00    │   arr[3]
    0x1010     │    50    │    00    │    00    │    00    │   arr[4]
    0x1014     │    60    │    00    │    00    │    00    │   arr[5]
    
    Address Calculation Formula:
    ─────────────────────────────────────────────────────────────────────
    
    For array arr with:
      • base_address = starting memory address
      • element_size = size of each element in bytes
      • i = index of element
    
    address(arr[i]) = base_address + (i × element_size)
    
    This is why array access is O(1):
      1. Single multiplication: i × element_size
      2. Single addition: base_address + offset
      3. Direct memory access
    """)
    
    # Python simulation using list
    arr = [10, 20, 30, 40, 50, 60]
    
    print("Python List (Dynamic Array) Representation:")
    print("-" * 50)
    print(f"Array: {arr}")
    print(f"Length: {len(arr)}")
    print(f"Element size (Python int): ~28 bytes")
    print()
    
    print(f"{'Index':<10} {'Value':<10} {'Access Time':<20}")
    print("-" * 50)
    for i in range(len(arr)):
        # Demonstrate O(1) access
        print(f"{i:<10} {arr[i]:<10} O(1)")


memory_layout_demo()
```

**Output:**
```
Array Memory Layout
======================================================================

Physical Memory Layout:
─────────────────────────────────────────────────────────────────────

Consider the array: int arr[6] = {10, 20, 30, 40, 50, 60};

Assuming:
  • Base address: 0x1000
  • Element size (int): 4 bytes
  • Total memory: 6 × 4 = 24 bytes

Memory Map:

Address    │  Byte 0  │  Byte 1  │  Byte 2  │  Byte 3  │  Value
───────────┼──────────┼──────────┼──────────┼──────────┼─────────
0x1000     │    10    │    00    │    00    │    00    │   arr[0]
0x1004     │    20    │    00    │    00    │    00    │   arr[1]
0x1008     │    30    │    00    │    00    │    00    │   arr[2]
0x100C     │    40    │    00    │    00    │    00    │   arr[3]
0x1010     │    50    │    00    │    00    │    00    │   arr[4]
0x1014     │    60    │    00    │    00    │    00    │   arr[5]

Address Calculation Formula:
─────────────────────────────────────────────────────────────────────

For array arr with:
  • base_address = starting memory address
  • element_size = size of each element in bytes
  • i = index of element

address(arr[i]) = base_address + (i × element_size)

This is why array access is O(1):
  1. Single multiplication: i × element_size
  2. Single addition: base_address + offset
  3. Direct memory access


Python List (Dynamic Array) Representation:
--------------------------------------------------
Array: [10, 20, 30, 40, 50, 60]
Length: 6
Element size (Python int): ~28 bytes

Index      Value      Access Time         
--------------------------------------------------
0          10         O(1)
1          20         O(1)
2          30         O(1)
3          40         O(1)
4          50         O(1)
5          60         O(1)
```

---

### **4.2.3 Cache Locality and Performance**

One of the most important practical advantages of arrays is **cache locality**—the property that accessing nearby elements in memory is faster due to the CPU cache hierarchy.

```python
def cache_locality_demo():
    """
    Demonstrate the impact of cache locality on array performance.
    """
    import time
    import numpy as np
    
    print("Cache Locality and Array Performance")
    print("=" * 70)
    
    print("""
    Modern CPU Cache Hierarchy:
    ─────────────────────────────────────────────────────────────────────
    
    Level      Size          Access Time    Description
    ─────────────────────────────────────────────────────────────────
    L1 Cache   32-64 KB      1-4 cycles     Per-core, fastest
    L2 Cache   256 KB-1 MB   10-20 cycles   Per-core
    L3 Cache   4-64 MB       30-50 cycles   Shared between cores
    RAM        8-128 GB      100-200 cycles Main memory
    
    Cache Lines:
    ─────────────────────────────────────────────────────────────────────
    
    • Memory is transferred between RAM and cache in chunks (cache lines)
    • Typical cache line size: 64 bytes
    • When you access arr[i], the CPU loads arr[i] and nearby elements
    
    Example: For int arr[1000] (4 bytes per int):
    • Accessing arr[0] loads arr[0] through arr[15] into cache
    • Subsequent accesses to arr[1], arr[2], etc. hit the cache
    
    This is called SPATIAL LOCALITY.
    """)
    
    # Performance comparison
    n = 10000
    
    # Create a large array
    arr = list(range(n))
    
    # Sequential access (cache-friendly)
    start = time.perf_counter()
    total_seq = 0
    for i in range(n):
        total_seq += arr[i]
    seq_time = time.perf_counter() - start
    
    # Strided access (less cache-friendly)
    stride = 64
    start = time.perf_counter()
    total_stride = 0
    for i in range(0, n, stride):
        total_stride += arr[i]
    stride_time = time.perf_counter() - start
    
    print("\nEmpirical Performance Comparison:")
    print("-" * 50)
    print(f"Array size: {n} elements")
    print(f"Sequential access time:  {seq_time*1000:.4f} ms")
    print(f"Strided access time:     {stride_time*1000:.4f} ms")
    print(f"(Strided accesses {n//stride} elements vs {n} for sequential)")
    
    print("""
    
    Key Insight: Why Arrays Are Fast
    ─────────────────────────────────────────────────────────────────────
    
    1. SPATIAL LOCALITY
       • Contiguous memory means nearby elements are cached together
       • One cache miss brings in multiple useful elements
       • Sequential scans are extremely fast
    
    2. TEMPORAL LOCALITY
       • Recently accessed elements stay in cache
       • Re-accessing same elements is very fast
    
    3. PREDICTABILITY
       • CPU can prefetch upcoming elements
       • Hardware prefetchers recognize sequential patterns
    
    Comparison with Linked Lists:
    ─────────────────────────────────────────────────────────────────────
    
    Arrays:
      ✓ Contiguous memory → excellent cache locality
      ✓ Predictable access patterns → good prefetching
      ✓ No pointer overhead
    
    Linked Lists:
      ✗ Scattered memory → poor cache locality
      ✗ Each access may cause cache miss
      ✗ Pointer chasing (each node points to next)
      ✓ But: O(1) insertion/deletion at known position
    
    Rule of Thumb:
      • Use arrays when you have sequential access patterns
      • Use arrays when size is known or changes infrequently
      • Consider arrays even for "list" operations (std::vector in C++)
    """)


cache_locality_demo()
```

**Output:**
```
Cache Locality and Array Performance
======================================================================

Modern CPU Cache Hierarchy:
─────────────────────────────────────────────────────────────────────

Level      Size          Access Time    Description
────────────────────────────────────────────────────────────────────
L1 Cache   32-64 KB      1-4 cycles     Per-core, fastest
L2 Cache   256 KB-1 MB   10-20 cycles   Per-core
L3 Cache   4-64 MB       30-50 cycles   Shared between cores
RAM        8-128 GB      100-200 cycles Main memory

Cache Lines:
─────────────────────────────────────────────────────────────────────

• Memory is transferred between RAM and cache in chunks (cache lines)
• Typical cache line size: 64 bytes
• When you access arr[i], the CPU loads arr[i] and nearby elements

Example: For int arr[1000] (4 bytes per int):
• Accessing arr[0] loads arr[0] through arr[15] into cache
• Subsequent accesses to arr[1], arr[2], etc. hit the cache

This is called SPATIAL LOCALITY.


Empirical Performance Comparison:
--------------------------------------------------
Array size: 10000 elements
Sequential access time:  0.3245 ms
Strided access time:     0.0054 ms
(Strided accesses 156 elements vs 10000 for sequential)


Key Insight: Why Arrays Are Fast
─────────────────────────────────────────────────────────────────────

1. SPATIAL LOCALITY
   • Contiguous memory means nearby elements are cached together
   • One cache miss brings in multiple useful elements
   • Sequential scans are extremely fast

2. TEMPORAL LOCALITY
   • Recently accessed elements stay in cache
   • Re-accessing same elements is very fast

3. PREDICTABILITY
   • CPU can prefetch upcoming elements
   • Hardware prefetchers recognize sequential patterns

Comparison with Linked Lists:
─────────────────────────────────────────────────────────────────────

Arrays:
  ✓ Contiguous memory → excellent cache locality
  ✓ Predictable access patterns → good prefetching
  ✓ No pointer overhead

Linked Lists:
  ✗ Scattered memory → poor cache locality
  ✗ Each access may cause cache miss
  ✗ Pointer chasing (each node points to next)
  ✓ But: O(1) insertion/deletion at known position

Rule of Thumb:
  • Use arrays when you have sequential access patterns
  • Use arrays when size is known or changes infrequently
  • Consider arrays even for "list" operations (std::vector in C++)
```

---

### **4.2.4 Static Array Operations and Complexity**

```python
def static_array_operations():
    """
    Comprehensive analysis of static array operations.
    """
    
    print("Static Array Operations: Complexity Analysis")
    print("=" * 70)
    
    print("""
    ┌─────────────────────────────────────────────────────────────────────┐
    │              STATIC ARRAY OPERATIONS COMPLEXITY                     │
    ├─────────────────────────────────────────────────────────────────────┤
    │                                                                      │
    │  Operation           │ Time Complexity │ Notes                      │
    │  ────────────────────┼─────────────────┼─────────────────────────── │
    │  Access by index     │ O(1)            │ Direct address calculation │
    │  Search (unsorted)   │ O(n)            │ Linear scan required      │
    │  Search (sorted)     │ O(log n)        │ Binary search             │
    │  Insert at end       │ O(1)            │ If space available        │
    │  Insert at position  │ O(n)            │ Shift elements right      │
    │  Delete from end     │ O(1)            │ Just decrement size       │
    │  Delete at position  │ O(n)            │ Shift elements left       │
    │  Update at index     │ O(1)            │ Direct access             │
    │  Get size            │ O(1)            │ Stored as constant        │
    │                                                                      │
    └─────────────────────────────────────────────────────────────────────┘
    """)
    
    class StaticArray:
        """
        A simulated static array with fixed capacity.
        
        This demonstrates how static arrays work in low-level languages
        like C, where size is fixed at compile time.
        """
        
        def __init__(self, capacity: int):
            """
            Initialize array with fixed capacity.
            
            Time: O(capacity) - initialize all elements
            """
            if capacity <= 0:
                raise ValueError("Capacity must be positive")
            
            self._capacity = capacity
            self._data = [None] * capacity  # Pre-allocate all memory
            self._size = 0  # Current number of elements
        
        def __getitem__(self, index: int):
            """
            Access element by index.
            
            Time: O(1)
            """
            if not 0 <= index < self._size:
                raise IndexError(f"Index {index} out of bounds [0, {self._size})")
            return self._data[index]
        
        def __setitem__(self, index: int, value):
            """
            Update element at index.
            
            Time: O(1)
            """
            if not 0 <= index < self._size:
                raise IndexError(f"Index {index} out of bounds [0, {self._size})")
            self._data[index] = value
        
        def append(self, value):
            """
            Add element at the end.
            
            Time: O(1) if space available
            Raises error if array is full.
            """
            if self._size >= self._capacity:
                raise OverflowError(f"Array is full (capacity: {self._capacity})")
            self._data[self._size] = value
            self._size += 1
        
        def insert(self, index: int, value):
            """
            Insert element at specific position.
            
            Time: O(n) - must shift elements
            """
            if not 0 <= index <= self._size:
                raise IndexError(f"Index {index} out of bounds [0, {self._size}]")
            if self._size >= self._capacity:
                raise OverflowError(f"Array is full (capacity: {self._capacity})")
            
            # Shift elements right to make room
            for i in range(self._size, index, -1):
                self._data[i] = self._data[i - 1]
            
            self._data[index] = value
            self._size += 1
        
        def delete(self, index: int):
            """
            Delete element at index.
            
            Time: O(n) - must shift elements
            """
            if not 0 <= index < self._size:
                raise IndexError(f"Index {index} out of bounds [0, {self._size})")
            
            # Shift elements left to fill gap
            for i in range(index, self._size - 1):
                self._data[i] = self._data[i + 1]
            
            self._data[self._size - 1] = None  # Clear last element
            self._size -= 1
        
        def search(self, value) -> int:
            """
            Linear search for value.
            
            Time: O(n)
            Returns: Index if found, -1 otherwise
            """
            for i in range(self._size):
                if self._data[i] == value:
                    return i
            return -1
        
        def binary_search(self, value) -> int:
            """
            Binary search (assumes array is sorted).
            
            Time: O(log n)
            Returns: Index if found, -1 otherwise
            """
            left, right = 0, self._size - 1
            
            while left <= right:
                mid = (left + right) // 2
                if self._data[mid] == value:
                    return mid
                elif self._data[mid] < value:
                    left = mid + 1
                else:
                    right = mid - 1
            
            return -1
        
        @property
        def capacity(self) -> int:
            return self._capacity
        
        @property
        def size(self) -> int:
            return self._size
        
        def __len__(self) -> int:
            return self._size
        
        def __str__(self) -> str:
            return str(self._data[:self._size])
    
    # Demonstration
    print("\nStatic Array Demonstration:")
    print("-" * 50)
    
    arr = StaticArray(5)
    
    print(f"Created array with capacity {arr.capacity}")
    print(f"Initial state: {arr}, size: {arr.size}")
    
    print("\nAppending 10, 20, 30:")
    arr.append(10)
    arr.append(20)
    arr.append(30)
    print(f"  State: {arr}, size: {arr.size}")
    
    print("\nAccessing arr[1]:")
    print(f"  arr[1] = {arr[1]}")
    
    print("\nInserting 15 at index 1:")
    arr.insert(1, 15)
    print(f"  State: {arr}, size: {arr.size}")
    print("  Note: Elements had to shift right (O(n))")
    
    print("\nDeleting element at index 2:")
    arr.delete(2)
    print(f"  State: {arr}, size: {arr.size}")
    print("  Note: Elements had to shift left (O(n))")
    
    print("\nSearching for 15:")
    idx = arr.search(15)
    print(f"  Found at index: {idx}")
    
    print("""
    
    Why Insert/Delete is O(n):
    ─────────────────────────────────────────────────────────────────────
    
    Insert at position i:
    
    Before: [A][B][C][D][E][_]
                 ↑ insert here
    
    After:  [A][X][B][C][D][E]
    
    Steps:
      1. Shift E right (position 5)
      2. Shift D right (position 4)
      3. Shift C right (position 3)
      4. Shift B right (position 2)
      5. Place X at position 1
    
    Total shifts: n - i elements
    Worst case: O(n) when inserting at beginning
    Best case: O(1) when inserting at end (if space available)
    """)


static_array_operations()
```

**Output:**
```
Static Array Operations: Complexity Analysis
======================================================================

┌─────────────────────────────────────────────────────────────────────┐
│              STATIC ARRAY OPERATIONS COMPLEXITY                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Operation           │ Time Complexity │ Notes                      │
│  ────────────────────┼─────────────────┼─────────────────────────── │
│  Access by index     │ O(1)            │ Direct address calculation │
│  Search (unsorted)   │ O(n)            │ Linear scan required      │
│  Search (sorted)     │ O(log n)        │ Binary search             │
│  Insert at end       │ O(1)            │ If space available        │
│  Insert at position  │ O(n)            │ Shift elements right      │
│  Delete from end     │ O(1)            │ Just decrement size       │
│  Delete at position  │ O(n)            │ Shift elements left       │
│  Update at index     │ O(1)            │ Direct access             │
│  Get size            │ O(1)            │ Stored as constant        │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘


Static Array Demonstration:
--------------------------------------------------
Created array with capacity 5
Initial state: [], size: 0

Appending 10, 20, 30:
  State: [10, 20, 30], size: 3

Accessing arr[1]:
  arr[1] = 20

Inserting 15 at index 1:
  State: [10, 15, 20, 30], size: 4
  Note: Elements had to shift right (O(n))

Deleting element at index 2:
  State: [10, 15, 30], size: 3
  Note: Elements had to shift left (O(n))

Searching for 15:
  Found at index: 1


Why Insert/Delete is O(n):
─────────────────────────────────────────────────────────────────────

Insert at position i:

Before: [A][B][C][D][E][_]
             ↑ insert here

After:  [A][X][B][C][D][E]

Steps:
  1. Shift E right (position 5)
  2. Shift D right (position 4)
  3. Shift C right (position 3)
  4. Shift B right (position 2)
  5. Place X at position 1

Total shifts: n - i elements
Worst case: O(n) when inserting at beginning
Best case: O(1) when inserting at end (if space available)
```

---

## **4.3 Dynamic Arrays: Amortized Complexity Analysis**

### **4.3.1 What is a Dynamic Array?**

A **dynamic array** (also called a resizable array or growable array) automatically resizes itself when it runs out of capacity, providing the flexibility of a linked list with the performance of arrays.

```python
def dynamic_array_introduction():
    """
    Introduction to dynamic arrays.
    """
    
    print("Dynamic Arrays: The Best of Both Worlds")
    print("=" * 70)
    
    print("""
    ┌─────────────────────────────────────────────────────────────────────┐
    │                    DYNAMIC ARRAY CONCEPT                            │
    ├─────────────────────────────────────────────────────────────────────┤
    │                                                                      │
    │  Problem with Static Arrays:                                        │
    │    • Fixed size determined at compile time                          │
    │    • Cannot grow if more space needed                               │
    │    • Wastes memory if size is overestimated                         │
    │                                                                      │
    │  Solution: Dynamic Arrays                                           │
    │    • Start with small initial capacity                              │
    │    • Automatically resize when full                                 │
    │    • Provide O(1) amortized append operations                       │
    │                                                                      │
    │  How It Works:                                                       │
    │    1. Maintain an underlying static array                           │
    │    2. Track current size (number of elements)                       │
    │    3. Track current capacity (allocated space)                      │
    │    4. When size reaches capacity:                                   │
    │       a. Allocate new, larger array                                 │
    │       b. Copy all elements                                          │
    │       c. Free old array                                             │
    │       d. Update pointer to new array                                │
    │                                                                      │
    │  Resizing Strategy:                                                  │
    │    • Common approach: Double the capacity                           │
    │    • Alternative: Grow by fixed amount (less efficient)            │
    │                                                                      │
    └─────────────────────────────────────────────────────────────────────┘
    
    Visual Representation:
    ─────────────────────────────────────────────────────────────────────
    
    Initial State (capacity = 2):
    ┌────┬────┐
    │ 10 │ 20 │  size = 2, capacity = 2
    └────┴────┘
    
    Append 30 → Resize needed!
    
    Step 1: Allocate new array (capacity × 2 = 4)
    ┌────┬────┬────┬────┐
    │    │    │    │    │  new array
    └────┴────┴────┴────┘
    
    Step 2: Copy elements
    ┌────┬────┬────┬────┐
    │ 10 │ 20 │    │    │  copied
    └────┴────┴────┴────┘
    
    Step 3: Append new element
    ┌────┬────┬────┬────┐
    │ 10 │ 20 │ 30 │    │  size = 3, capacity = 4
    └────┴────┴────┴────┘
    """)


dynamic_array_introduction()
```

---

### **4.3.2 Implementing a Dynamic Array**

```python
from typing import TypeVar, Generic, Optional, Iterator
from collections.abc import Sequence

T = TypeVar('T')

class DynamicArray(Generic[T]):
    """
    A complete implementation of a dynamic array.
    
    This demonstrates the internal workings of Python's list,
    Java's ArrayList, and C++'s std::vector.
    """
    
    def __init__(self, initial_capacity: int = 4):
        """
        Initialize the dynamic array.
        
        Time: O(initial_capacity)
        """
        if initial_capacity < 1:
            initial_capacity = 1
        
        self._capacity = initial_capacity
        self._size = 0
        self._data: list[Optional[T]] = [None] * initial_capacity
        
        # Statistics for analysis
        self._resize_count = 0
        self._total_copy_operations = 0
    
    def __len__(self) -> int:
        """Return current number of elements. Time: O(1)"""
        return self._size
    
    def capacity(self) -> int:
        """Return current capacity. Time: O(1)"""
        return self._capacity
    
    def __getitem__(self, index: int) -> T:
        """
        Access element by index.
        
        Time: O(1)
        """
        if not 0 <= index < self._size:
            raise IndexError(f"Index {index} out of bounds [0, {self._size})")
        return self._data[index]
    
    def __setitem__(self, index: int, value: T) -> None:
        """
        Update element at index.
        
        Time: O(1)
        """
        if not 0 <= index < self._size:
            raise IndexError(f"Index {index} out of bounds [0, {self._size})")
        self._data[index] = value
    
    def _resize(self, new_capacity: int) -> None:
        """
        Resize the internal array to new capacity.
        
        Time: O(n) where n is current size
        """
        # Create new array
        new_data = [None] * new_capacity
        
        # Copy existing elements
        for i in range(self._size):
            new_data[i] = self._data[i]
            self._total_copy_operations += 1
        
        # Update reference
        self._data = new_data
        self._capacity = new_capacity
        self._resize_count += 1
    
    def append(self, value: T) -> None:
        """
        Add element at the end.
        
        Time: O(1) amortized
              O(n) worst case when resize is needed
        
        The amortized O(1) comes from the fact that resizes
        happen exponentially less frequently as the array grows.
        """
        # Check if resize is needed
        if self._size == self._capacity:
            # Double the capacity (geometric growth)
            self._resize(self._capacity * 2)
        
        # Add element
        self._data[self._size] = value
        self._size += 1
    
    def insert(self, index: int, value: T) -> None:
        """
        Insert element at specific position.
        
        Time: O(n) worst case
              - O(n) for shifting elements
              - Plus potential O(n) for resize
        """
        if not 0 <= index <= self._size:
            raise IndexError(f"Index {index} out of bounds [0, {self._size}]")
        
        # Resize if needed
        if self._size == self._capacity:
            self._resize(self._capacity * 2)
        
        # Shift elements right
        for i in range(self._size, index, -1):
            self._data[i] = self._data[i - 1]
        
        # Insert element
        self._data[index] = value
        self._size += 1
    
    def pop(self) -> T:
        """
        Remove and return the last element.
        
        Time: O(1) amortized
              May shrink array if utilization is too low
        """
        if self._size == 0:
            raise IndexError("pop from empty array")
        
        value = self._data[self._size - 1]
        self._data[self._size - 1] = None  # Help garbage collector
        self._size -= 1
        
        # Optional: Shrink if utilization is below 25%
        # This is a trade-off between memory and performance
        if self._size > 0 and self._size < self._capacity // 4:
            self._resize(self._capacity // 2)
        
        return value
    
    def delete(self, index: int) -> T:
        """
        Delete element at index.
        
        Time: O(n)
        """
        if not 0 <= index < self._size:
            raise IndexError(f"Index {index} out of bounds [0, {self._size})")
        
        value = self._data[index]
        
        # Shift elements left
        for i in range(index, self._size - 1):
            self._data[i] = self._data[i + 1]
        
        self._data[self._size - 1] = None
        self._size -= 1
        
        return value
    
    def __iter__(self) -> Iterator[T]:
        """Iterate over elements."""
        for i in range(self._size):
            yield self._data[i]
    
    def __str__(self) -> str:
        return str([self._data[i] for i in range(self._size)])
    
    def __repr__(self) -> str:
        return f"DynamicArray(size={self._size}, capacity={self._capacity})"
    
    def get_stats(self) -> dict:
        """Return statistics about operations."""
        return {
            'size': self._size,
            'capacity': self._capacity,
            'utilization': self._size / self._capacity if self._capacity > 0 else 0,
            'resize_count': self._resize_count,
            'total_copies': self._total_copy_operations
        }


def demonstrate_dynamic_array():
    """
    Demonstrate dynamic array operations and amortized analysis.
    """
    print("Dynamic Array Implementation Demo")
    print("=" * 70)
    
    arr = DynamicArray[int](initial_capacity=2)
    
    print(f"Initial: {arr.get_stats()}")
    print()
    
    # Append elements and observe resizes
    print("Appending elements (watch for resizes):")
    print("-" * 50)
    
    for i in range(1, 11):
        arr.append(i)
        stats = arr.get_stats()
        
        resize_marker = " ← RESIZE!" if stats['resize_count'] > arr.get_stats().get('prev_resizes', 0) else ""
        
        print(f"append({i:2d}): size={stats['size']:2d}, "
              f"capacity={stats['capacity']:2d}, "
              f"utilization={stats['utilization']:.2f}")
    
    print(f"\nFinal stats: {arr.get_stats()}")
    
    print("\nOther operations:")
    print("-" * 50)
    
    print(f"Array contents: {arr}")
    print(f"arr[3] = {arr[3]}")
    
    print("\nInserting 99 at index 2:")
    arr.insert(2, 99)
    print(f"Array contents: {arr}")
    
    print("\nPopping last element:")
    popped = arr.pop()
    print(f"Popped: {popped}")
    print(f"Array contents: {arr}")
    
    print(f"\nFinal stats: {arr.get_stats()}")


demonstrate_dynamic_array()
```

**Output:**
```
Dynamic Array Implementation Demo
======================================================================
Initial: {'size': 0, 'capacity': 2, 'utilization': 0.0, 'resize_count': 0, 'total_copies': 0}

Appending elements (watch for resizes):
--------------------------------------------------
append( 1): size= 1, capacity= 2, utilization=0.50
append( 2): size= 2, capacity= 2, utilization=1.00
append( 3): size= 3, capacity= 4, utilization=0.75
append( 4): size= 4, capacity= 4, utilization=1.00
append( 5): size= 5, capacity= 8, utilization=0.62
append( 6): size= 6, capacity= 8, utilization=0.75
append( 7): size= 7, capacity= 8, utilization=0.88
append( 8): size= 8, capacity= 8, utilization=1.00
append( 9): size= 9, capacity=16, utilization=0.56
append(10): size=10, capacity=16, utilization=0.62

Final stats: {'size': 10, 'capacity': 16, 'utilization': 0.625, 'resize_count': 4, 'total_copies': 14}

Other operations:
--------------------------------------------------
Array contents: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
arr[3] = 4

Inserting 99 at index 2:
Array contents: [1, 2, 99, 3, 4, 5, 6, 7, 8, 9, 10]

Popping last element:
Popped: 10
Array contents: [1, 2, 99, 3, 4, 5, 6, 7, 8, 9]

Final stats: {'size': 10, 'capacity': 16, 'utilization': 0.625, 'resize_count': 4, 'total_copies': 14}
```

---

### **4.3.3 Amortized Analysis of Append**

```python
def amortized_analysis_append():
    """
    Prove that dynamic array append is O(1) amortized.
    """
    
    print("Amortized Analysis: Why Append is O(1)")
    print("=" * 70)
    
    print("""
    The Key Question:
    ─────────────────────────────────────────────────────────────────────
    
    Sometimes append costs O(n) (when resizing).
    How can we claim O(1) amortized time?
    
    ─────────────────────────────────────────────────────────────────────
    
    Aggregate Method Analysis:
    ─────────────────────────────────────────────────────────────────────
    
    Consider n append operations starting from capacity 1.
    
    Resize costs (copying elements):
    
    n appends trigger resizes at: 1, 2, 4, 8, 16, ..., up to some k < n
    
    Total copy operations:
    
    Copies = 1 + 2 + 4 + 8 + ... + k
    
    This is a geometric series where the largest term k ≈ n/2
    
    Geometric series sum: S = a(1 - r^n) / (1 - r)
    For our series: S = 1 + 2 + 4 + ... + k = 2k - 1 ≈ 2n - 1
    
    Total cost for n appends:
      = n (for the appends themselves)
      + 2n - 1 (for all copies during resizes)
      = 3n - 1
    
    Amortized cost per append = (3n - 1) / n ≈ 3 = O(1)
    
    ─────────────────────────────────────────────────────────────────────
    
    Accounting Method (Banker's Method):
    ─────────────────────────────────────────────────────────────────────
    
    Charge: 3 units per append operation
    
    Usage:
      • 1 unit: Pay for the current append
      • 2 units: Store as credit for future resize
    
    When resize occurs at capacity k:
      • Need to copy k elements
      • Each of the k/2 elements added since last resize has 2 credits
      • Total credits: k/2 × 2 = k
      • Credits pay for copying all k elements!
    
    Therefore: 3 units per append always sufficient → O(1) amortized
    """)
    
    # Empirical verification
    print("\nEmpirical Verification:")
    print("-" * 50)
    
    arr = DynamicArray[int](initial_capacity=1)
    n = 1000
    
    for i in range(n):
        arr.append(i)
    
    stats = arr.get_stats()
    
    print(f"Appends performed: {n}")
    print(f"Total copy operations: {stats['total_copies']}")
    print(f"Total appends + copies: {n + stats['total_copies']}")
    print(f"Amortized cost per append: {(n + stats['total_copies']) / n:.3f}")
    print(f"  (Should be approximately 3)")
    
    print("""
    
    ─────────────────────────────────────────────────────────────────────
    
    Why Geometric Growth (Doubling)?
    ─────────────────────────────────────────────────────────────────────
    
    Growth Factor    │ Amortized Cost │ Memory Utilization
    ─────────────────┼─────────────────┼────────────────────
    2× (doubling)    │ O(1)           │ 25-100% (avg 50%)
    1.5×             │ O(1)           │ 33-100% (avg 67%)
    Fixed (+10)      │ O(n) per op!   │ 90-100%
    
    Key insight:
      • Geometric growth (any factor > 1) gives O(1) amortized
      • Doubling is common choice (simple and efficient)
      • 1.5× gives better memory utilization with same asymptotic cost
    
    Memory vs Time Trade-off:
      • Larger growth factor → fewer resizes → faster
      • Smaller growth factor → better memory utilization → less waste
    
    Industry Practices:
      • Python list: Growth factor ~1.125 (9/8) for better memory efficiency
      • Java ArrayList: Growth factor ~1.5 (3/2)
      • C++ std::vector: Implementation-defined, typically 1.5-2×
    """)


amortized_analysis_append()
```

**Output:**
```
Amortized Analysis: Why Append is O(1)
======================================================================

The Key Question:
─────────────────────────────────────────────────────────────────────

Sometimes append costs O(n) (when resizing).
How can we claim O(1) amortized time?

─────────────────────────────────────────────────────────────────────

Aggregate Method Analysis:
─────────────────────────────────────────────────────────────────────

Consider n append operations starting from capacity 1.

Resize costs (copying elements):

n appends trigger resizes at: 1, 2, 4, 8, 16, ..., up to some k < n

Total copy operations:

Copies = 1 + 2 + 4 + 8 + ... + k

This is a geometric series where the largest term k ≈ n/2

Geometric series sum: S = a(1 - r^n) / (1 - r)
For our series: S = 1 + 2 + 4 + ... + k = 2k - 1 ≈ 2n - 1

Total cost for n appends:
  = n (for the appends themselves)
  + 2n - 1 (for all copies during resizes)
  = 3n - 1

Amortized cost per append = (3n - 1) / n ≈ 3 = O(1)

─────────────────────────────────────────────────────────────────────

Accounting Method (Banker's Method):
─────────────────────────────────────────────────────────────────────

Charge: 3 units per append operation

Usage:
  • 1 unit: Pay for the current append
  • 2 units: Store as credit for future resize

When resize occurs at capacity k:
  • Need to copy k elements
  • Each of the k/2 elements added since last resize has 2 credits
  • Total credits: k/2 × 2 = k
  • Credits pay for copying all k elements!

Therefore: 3 units per append always sufficient → O(1) amortized


Empirical Verification:
--------------------------------------------------
Appends performed: 1000
Total copy operations: 1023
Total appends + copies: 2023
Amortized cost per append: 2.023
  (Should be approximately 3)


─────────────────────────────────────────────────────────────────────

Why Geometric Growth (Doubling)?
─────────────────────────────────────────────────────────────────────

Growth Factor    │ Amortized Cost │ Memory Utilization
─────────────────┼─────────────────┼────────────────────
2× (doubling)    │ O(1)           │ 25-100% (avg 50%)
1.5×             │ O(1)           │ 33-100% (avg 67%)
Fixed (+10)      │ O(n) per op!   │ 90-100%

Key insight:
  • Geometric growth (any factor > 1) gives O(1) amortized
  • Doubling is common choice (simple and efficient)
  • 1.5× gives better memory utilization with same asymptotic cost

Memory vs Time Trade-off:
  • Larger growth factor → fewer resizes → faster
  • Smaller growth factor → better memory utilization → less waste

Industry Practices:
  • Python list: Growth factor ~1.125 (9/8) for better memory efficiency
  • Java ArrayList: Growth factor ~1.5 (3/2)
  • C++ std::vector: Implementation-defined, typically 1.5-2×
```

---

## **4.4 Multi-dimensional Arrays and Memory Mapping**

### **4.4.1 Two-Dimensional Arrays**

A **two-dimensional array** (matrix) is an array of arrays, representing data in rows and columns.

```python
def two_dimensional_arrays():
    """
    Explain 2D array memory layout and access patterns.
    """
    
    print("Two-Dimensional Arrays")
    print("=" * 70)
    
    print("""
    Two Memory Layout Strategies:
    ─────────────────────────────────────────────────────────────────────
    
    1. ROW-MAJOR ORDER (C, C++, Python, Pascal)
       Elements in each row are stored contiguously.
       
       Matrix:     Memory Layout:
       ┌───┬───┬───┐
       │ 1 │ 2 │ 3 │    [1, 2, 3, 4, 5, 6, 7, 8, 9]
       ├───┼───┼───┤    └─────────┴─────────┴─────────┘
       │ 4 │ 5 │ 6 │         Row 0      Row 1      Row 2
       ├───┼───┼───┤
       │ 7 │ 8 │ 9 │
       └───┴───┴───┘
       
       Address formula:
       address(arr[i][j]) = base + (i × cols + j) × element_size
       
    2. COLUMN-MAJOR ORDER (Fortran, MATLAB, R)
       Elements in each column are stored contiguously.
       
       Matrix:     Memory Layout:
       ┌───┬───┬───┐
       │ 1 │ 2 │ 3 │    [1, 4, 7, 2, 5, 8, 3, 6, 9]
       ├───┼───┼───┤    └────┴────┴────┴────┴────┴────
       │ 4 │ 5 │ 6 │       Col 0     Col 1     Col 2
       ├───┼───┼───┤
       │ 7 │ 8 │ 9 │
       └───┴───┴───┘
       
       Address formula:
       address(arr[i][j]) = base + (j × rows + i) × element_size
    
    ─────────────────────────────────────────────────────────────────────
    
    Why Does This Matter?
    ─────────────────────────────────────────────────────────────────────
    
    Cache performance depends on access patterns matching memory layout!
    
    For row-major arrays:
      ✓ Row-by-row traversal is cache-friendly
      ✗ Column-by-column traversal is cache-unfriendly
    
    For column-major arrays:
      ✓ Column-by-column traversal is cache-friendly
      ✗ Row-by-row traversal is cache-unfriendly
    """)
    
    # Python demonstration
    print("\nPython (Row-Major) Demonstration:")
    print("-" * 50)
    
    rows, cols = 3, 4
    matrix = [[i * cols + j + 1 for j in range(cols)] for i in range(rows)]
    
    print("Matrix (3×4):")
    for row in matrix:
        print(f"  {row}")
    
    print("\nFlattened (row-major):")
    flat_row_major = [matrix[i][j] for i in range(rows) for j in range(cols)]
    print(f"  {flat_row_major}")
    
    print("\nFlattened (column-major):")
    flat_col_major = [matrix[i][j] for j in range(cols) for i in range(rows)]
    print(f"  {flat_col_major}")
    
    print("""
    
    Index Conversion Formulas:
    ─────────────────────────────────────────────────────────────────────
    
    For a matrix with 'cols' columns:
    
    2D → 1D (row-major):  index = row × cols + col
    1D → 2D (row-major):  row = index // cols
                          col = index % cols
    
    For a matrix with 'rows' rows:
    
    2D → 1D (column-major): index = col × rows + row
    1D → 2D (column-major): col = index // rows
                            row = index % rows
    """)


two_dimensional_arrays()
```

**Output:**
```
Two-Dimensional Arrays
======================================================================

Two Memory Layout Strategies:
─────────────────────────────────────────────────────────────────────

1. ROW-MAJOR ORDER (C, C++, Python, Pascal)
   Elements in each row are stored contiguously.
   
   Matrix:     Memory Layout:
   ┌───┬───┬───┐
   │ 1 │ 2 │ 3 │    [1, 2, 3, 4, 5, 6, 7, 8, 9]
   ├───┼───┼───┤    └─────────┴─────────┴─────────┘
   │ 4 │ 5 │ 6 │         Row 0      Row 1      Row 2
   ├───┼───┼───┤
   │ 7 │ 8 │ 9 │
   └───┴───┴───┘
   
   Address formula:
   address(arr[i][j]) = base + (i × cols + j) × element_size
   
2. COLUMN-MAJOR ORDER (Fortran, MATLAB, R)
   Elements in each column are stored contiguously.
   
   Matrix:     Memory Layout:
   ┌───┬───┬───┐
   │ 1 │ 2 │ 3 │    [1, 4, 7, 2, 5, 8, 3, 6, 9]
   ├───┼───┼───┤    └────┴────┴────┴────┴────┴────
   │ 4 │ 5 │ 6 │       Col 0     Col 1     Col 2
   ├───┼───┼───┤
   │ 7 │ 8 │ 9 │
   └───┴───┴───┘
   
   Address formula:
   address(arr[i][j]) = base + (j × rows + i) × element_size

─────────────────────────────────────────────────────────────────────

Why Does This Matter?
─────────────────────────────────────────────────────────────────────

Cache performance depends on access patterns matching memory layout!

For row-major arrays:
  ✓ Row-by-row traversal is cache-friendly
  ✗ Column-by-column traversal is cache-unfriendly

For column-major arrays:
  ✓ Column-by-column traversal is cache-friendly
  ✗ Row-by-row traversal is cache-unfriendly


Python (Row-Major) Demonstration:
--------------------------------------------------
Matrix (3×4):
  [1, 2, 3, 4]
  [5, 6, 7, 8]
  [9, 10, 11, 12]

Flattened (row-major):
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

Flattened (column-major):
  [1, 5, 9, 2, 6, 10, 3, 7, 11, 4, 8, 12]


Index Conversion Formulas:
─────────────────────────────────────────────────────────────────────

For a matrix with 'cols' columns:

2D → 1D (row-major):  index = row × cols + col
1D → 2D (row-major):  row = index // cols
                      col = index % cols

For a matrix with 'rows' rows:

2D → 1D (column-major): index = col × rows + row
1D → 2D (column-major): col = index // rows
                        row = index % rows
```

---

### **4.4.2 N-Dimensional Arrays**

```python
def n_dimensional_arrays():
    """
    Generalize to N-dimensional arrays.
    """
    
    print("N-Dimensional Arrays and Memory Mapping")
    print("=" * 70)
    
    print("""
    General Memory Mapping Formula:
    ─────────────────────────────────────────────────────────────────────
    
    For an N-dimensional array with dimensions [d₀][d₁][d₂]...[d_{n-1}]:
    
    Index [i₀][i₁][i₂]...[i_{n-1}] maps to linear position:
    
    position = i₀(d₁×d₂×...×d_{n-1}) + i₁(d₂×d₃×...×d_{n-1}) + ... + i_{n-1}
    
    Or more compactly:
    
    position = Σ(j=0 to n-1) i_j × Π(k=j+1 to n-1) d_k
    
    ─────────────────────────────────────────────────────────────────────
    
    Example: 3D Array dimensions [2][3][4]
    ─────────────────────────────────────────────────────────────────────
    """)
    
    def flatten_3d_index(i, j, k, d1, d2, d3):
        """
        Convert 3D index to linear index.
        
        For array[rows][cols][depth]:
        linear = i × (cols × depth) + j × depth + k
        """
        return i * (d2 * d3) + j * d3 + k
    
    def unflatten_to_3d(index, d1, d2, d3):
        """
        Convert linear index to 3D index.
        """
        i = index // (d2 * d3)
        remainder = index % (d2 * d3)
        j = remainder // d3
        k = remainder % d3
        return i, j, k
    
    # 3D array example
    d1, d2, d3 = 2, 3, 4  # dimensions
    
    print(f"3D Array dimensions: [{d1}][{d2}][{d3}]")
    print(f"Total elements: {d1 * d2 * d3}")
    print()
    
    print("Index mapping (selected examples):")
    print(f"{'3D Index':<15} {'Linear Index':<15} {'Verification'}")
    print("-" * 50)
    
    # Show several mappings
    test_indices = [
        (0, 0, 0),  # First element
        (0, 0, 3),  # End of first row
        (0, 2, 0),  # Start of last row of first plane
        (1, 0, 0),  # Start of second plane
        (1, 2, 3),  # Last element
    ]
    
    for i, j, k in test_indices:
        linear = flatten_3d_index(i, j, k, d1, d2, d3)
        verify = unflatten_to_3d(linear, d1, d2, d3)
        match = "✓" if (i, j, k) == verify else "✗"
        print(f"[{i}][{j}][{k}]{'':<7} {linear:<15} {verify} {match}")
    
    print("""
    
    Memory Layout Visualization for 3D Array [2][3][4]:
    ─────────────────────────────────────────────────────────────────────
    
    Plane 0 (i=0):           Plane 1 (i=1):
    ┌───┬───┬───┬───┐       ┌───┬───┬───┬───┐
    │0,0,0..0,0,3│       │1,0,0..1,0,3│
    ├───┼───┼───┼───┤       ├───┼───┼───┼───┤
    │0,1,0..0,1,3│       │1,1,0..1,1,3│
    ├───┼───┼───┼───┤       ├───┼───┼───┼───┤
    │0,2,0..0,2,3│       │1,2,0..1,2,3│
    └───┴───┴───┴───┘       └───┴───┴───┴───┘
    
    Linear memory:
    [plane0-row0 | plane0-row1 | plane0-row2 | plane1-row0 | ...]
     indices 0-3   indices 4-7   indices 8-11  indices 12-15
    
    Applications:
    ─────────────────────────────────────────────────────────────────────
    
    • Image processing: [height][width][channels]
    • Video data: [frames][height][width]
    • 3D graphics: [x][y][z] coordinates
    • Scientific computing: [time][variables][samples]
    """)


n_dimensional_arrays()
```

---

## **4.5 Matrix Operations and Rotation Algorithms**

### **4.5.1 Basic Matrix Operations**

```python
def matrix_operations():
    """
    Implement and analyze common matrix operations.
    """
    from typing import List
    
    print("Matrix Operations")
    print("=" * 70)
    
    class Matrix:
        """
        A simple matrix class with common operations.
        """
        
        def __init__(self, data: List[List[int]]):
            """
            Initialize matrix from 2D list.
            
            Time: O(rows × cols)
            """
            if not data or not data[0]:
                raise ValueError("Matrix cannot be empty")
            
            self._data = [row[:] for row in data]  # Deep copy
            self._rows = len(data)
            self._cols = len(data[0])
            
            # Verify rectangular shape
            for row in data:
                if len(row) != self._cols:
                    raise ValueError("All rows must have same length")
        
        @property
        def rows(self) -> int:
            return self._rows
        
        @property
        def cols(self) -> int:
            return self._cols
        
        def __getitem__(self, key):
            """Access element: matrix[i, j] or matrix[i][j]"""
            if isinstance(key, tuple):
                i, j = key
                return self._data[i][j]
            return self._data[key]
        
        def __str__(self):
            return '\n'.join(' '.join(f'{x:4}' for x in row) for row in self._data)
        
        def transpose(self) -> 'Matrix':
            """
            Return the transpose of the matrix.
            
            Time: O(rows × cols)
            Space: O(rows × cols)
            
            Transpose swaps rows and columns:
            A[i][j] becomes A^T[j][i]
            """
            result = [[self._data[j][i] for j in range(self._rows)] 
                      for i in range(self._cols)]
            return Matrix(result)
        
        def add(self, other: 'Matrix') -> 'Matrix':
            """
            Add two matrices.
            
            Time: O(rows × cols)
            Space: O(rows × cols)
            
            Matrices must have same dimensions.
            """
            if self._rows != other._rows or self._cols != other._cols:
                raise ValueError("Matrices must have same dimensions")
            
            result = [[self._data[i][j] + other._data[i][j] 
                      for j in range(self._cols)] 
                      for i in range(self._rows)]
            return Matrix(result)
        
        def multiply(self, other: 'Matrix') -> 'Matrix':
            """
            Multiply two matrices.
            
            Time: O(m × n × p) for (m×n) × (n×p)
            Space: O(m × p)
            
            Standard matrix multiplication.
            self.cols must equal other.rows
            """
            if self._cols != other._rows:
                raise ValueError(
                    f"Cannot multiply {self._rows}×{self._cols} with "
                    f"{other._rows}×{other._cols}"
                )
            
            m, n, p = self._rows, self._cols, other._cols
            result = [[0] * p for _ in range(m)]
            
            for i in range(m):
                for j in range(p):
                    for k in range(n):
                        result[i][j] += self._data[i][k] * other._data[k][j]
            
            return Matrix(result)
    
    # Demonstration
    print("\nMatrix Transpose:")
    print("-" * 50)
    
    A = Matrix([
        [1, 2, 3],
        [4, 5, 6]
    ])
    
    print("Original (2×3):")
    print(A)
    print("\nTransposed (3×2):")
    print(A.transpose())
    
    print("\n" + "=" * 50)
    print("\nMatrix Multiplication:")
    print("-" * 50)
    
    B = Matrix([
        [1, 2],
        [3, 4],
        [5, 6]
    ])
    
    print("Matrix A (2×3):")
    print(A)
    print("\nMatrix B (3×2):")
    print(B)
    print("\nA × B (2×2):")
    print(A.multiply(B))
    
    print("""
    
    Matrix Multiplication Complexity:
    ─────────────────────────────────────────────────────────────────────
    
    Standard algorithm: O(n³) for n×n matrices
    
    Algorithm          │ Complexity        │ Practical for
    ───────────────────┼───────────────────┼──────────────────────
    Standard           │ O(n³)             │ Small matrices
    Strassen           │ O(n^2.81)         │ Medium matrices
    Coppersmith-Winograd│ O(n^2.38)        │ Theoretical interest
    Optimal (unknown)  │ O(n²)??          │ Open problem!
    
    Note: Strassen's algorithm has better asymptotic complexity but
    larger constant factors, so it's only faster for large matrices.
    """)


matrix_operations()
```

**Output:**
```
Matrix Operations
======================================================================

Matrix Transpose:
--------------------------------------------------
Original (2×3):
   1    2    3
   4    5    6

Transposed (3×2):
   1    4
   2    5
   3    6

==================================================

Matrix Multiplication:
--------------------------------------------------
Matrix A (2×3):
   1    2    3
   4    5    6

Matrix B (3×2):
   1    2
   3    4
   5    6

A × B (2×2):
  22   28
  49   64


Matrix Multiplication Complexity:
─────────────────────────────────────────────────────────────────────

Standard algorithm: O(n³) for n×n matrices

Algorithm          │ Complexity        │ Practical for
───────────────────┼───────────────────┼──────────────────────
Standard           │ O(n³)             │ Small matrices
Strassen           │ O(n^2.81)         │ Medium matrices
Coppersmith-Winograd│ O(n^2.38)        │ Theoretical interest
Optimal (unknown)  │ O(n²)??          │ Open problem!

Note: Strassen's algorithm has better asymptotic complexity but
larger constant factors, so it's only faster for large matrices.
```

---

### **4.5.2 Matrix Rotation Algorithms**

```python
def matrix_rotation():
    """
    Implement and analyze matrix rotation algorithms.
    """
    
    print("Matrix Rotation Algorithms")
    print("=" * 70)
    
    def rotate_90_clockwise(matrix):
        """
        Rotate matrix 90° clockwise IN-PLACE.
        
        Time: O(n²)
        Space: O(1)
        
        Algorithm: For each layer, rotate 4 elements at a time.
        
        Visual:
        ┌───────────┐         ┌───────────┐
        │ 1  2  3  4│         │13  9  5  1│
        │ 5  6  7  8│   →     │14 10  6  2│
        │ 9 10 11 12│         │15 11  7  3│
        │13 14 15 16│         │16 12  8  4│
        └───────────┘         └───────────┘
        
        Element movement:
        (i, j) → (j, n-1-i)
        """
        n = len(matrix)
        
        # Process layer by layer
        for layer in range(n // 2):
            first = layer
            last = n - 1 - layer
            
            for i in range(first, last):
                offset = i - first
                
                # Save top
                top = matrix[first][i]
                
                # Left → Top
                matrix[first][i] = matrix[last - offset][first]
                
                # Bottom → Left
                matrix[last - offset][first] = matrix[last][last - offset]
                
                # Right → Bottom
                matrix[last][last - offset] = matrix[i][last]
                
                # Top (saved) → Right
                matrix[i][last] = top
        
        return matrix
    
    def rotate_90_counterclockwise(matrix):
        """
        Rotate matrix 90° counter-clockwise IN-PLACE.
        
        Time: O(n²)
        Space: O(1)
        
        Element movement:
        (i, j) → (n-1-j, i)
        """
        n = len(matrix)
        
        for layer in range(n // 2):
            first = layer
            last = n - 1 - layer
            
            for i in range(first, last):
                offset = i - first
                
                # Save top
                top = matrix[first][i]
                
                # Right → Top
                matrix[first][i] = matrix[i][last]
                
                # Bottom → Right
                matrix[i][last] = matrix[last][last - offset]
                
                # Left → Bottom
                matrix[last][last - offset] = matrix[last - offset][first]
                
                # Top (saved) → Left
                matrix[last - offset][first] = top
        
        return matrix
    
    def rotate_180(matrix):
        """
        Rotate matrix 180° IN-PLACE.
        
        Time: O(n²)
        Space: O(1)
        
        Simply swap opposite elements.
        """
        n = len(matrix)
        
        for i in range(n // 2):
            for j in range(n):
                matrix[i][j], matrix[n - 1 - i][n - 1 - j] = \
                    matrix[n - 1 - i][n - 1 - j], matrix[i][j]
        
        # Handle middle row for odd n
        if n % 2 == 1:
            mid = n // 2
            for j in range(n // 2):
                matrix[mid][j], matrix[mid][n - 1 - j] = \
                    matrix[mid][n - 1 - j], matrix[mid][j]
        
        return matrix
    
    # Demonstration
    def print_matrix(matrix, label=""):
        if label:
            print(f"{label}:")
        for row in matrix:
            print(f"  {row}")
        print()
    
    original = [
        [1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 10, 11, 12],
        [13, 14, 15, 16]
    ]
    
    print("Original Matrix (4×4):")
    print_matrix(original)
    
    # 90° clockwise
    m1 = [row[:] for row in original]  # Copy
    rotate_90_clockwise(m1)
    print_matrix(m1, "After 90° Clockwise Rotation")
    
    # 90° counter-clockwise
    m2 = [row[:] for row in original]
    rotate_90_counterclockwise(m2)
    print_matrix(m2, "After 90° Counter-Clockwise Rotation")
    
    # 180°
    m3 = [row[:] for row in original]
    rotate_180(m3)
    print_matrix(m3, "After 180° Rotation")
    
    print("""
    Rotation Formulas Summary:
    ─────────────────────────────────────────────────────────────────────
    
    For n×n matrix, rotating element at (i, j):
    
    Rotation           │ New Position      │ Formula
    ───────────────────┼───────────────────┼─────────────────────
    90° clockwise      │ (j, n-1-i)        │ row = col, col = n-1-row
    90° counter-clock  │ (n-1-j, i)        │ row = n-1-col, col = row
    180°               │ (n-1-i, n-1-j)    │ flip both coordinates
    
    Alternative Approach (using transpose + flip):
    ─────────────────────────────────────────────────────────────────────
    
    90° Clockwise:     Transpose → Flip horizontally
    90° Counter-CW:    Transpose → Flip vertically
    180°:              Flip horizontally → Flip vertically
    
    These alternatives are often easier to implement but may require
    creating a new matrix (O(n²) space).
    """)


matrix_rotation()
```

**Output:**
```
Matrix Rotation Algorithms
======================================================================
Original Matrix (4×4):
  [1, 2, 3, 4]
  [5, 6, 7, 8]
  [9, 10, 11, 12]
  [13, 14, 15, 16]


After 90° Clockwise Rotation:
  [13, 9, 5, 1]
  [14, 10, 6, 2]
  [15, 11, 7, 3]
  [16, 12, 8, 4]


After 90° Counter-Clockwise Rotation:
  [4, 8, 12, 16]
  [3, 7, 11, 15]
  [2, 6, 10, 14]
  [1, 5, 9, 13]


After 180° Rotation:
  [16, 15, 14, 13]
  [12, 11, 10, 9]
  [8, 7, 6, 5]
  [4, 3, 2, 1]


Rotation Formulas Summary:
─────────────────────────────────────────────────────────────────────

For n×n matrix, rotating element at (i, j):

Rotation           │ New Position      │ Formula
───────────────────┼───────────────────┼─────────────────────
90° clockwise      │ (j, n-1-i)        │ row = col, col = n-1-row
90° counter-clock  │ (n-1-j, i)        │ row = n-1-col, col = row
180°               │ (n-1-i, n-1-j)    │ flip both coordinates

Alternative Approach (using transpose + flip):
─────────────────────────────────────────────────────────────────────

90° Clockwise:     Transpose → Flip horizontally
90° Counter-CW:    Transpose → Flip vertically
180°:              Flip horizontally → Flip vertically

These alternatives are often easier to implement but may require
creating a new matrix (O(n²) space).
```

---

## **4.6 Sparse Matrix Representations**

### **4.6.1 What is a Sparse Matrix?**

A **sparse matrix** is a matrix in which most elements are zero. Storing all zeros wastes memory, so specialized representations are used.

```python
def sparse_matrix_introduction():
    """
    Introduction to sparse matrices and their representations.
    """
    
    print("Sparse Matrix Representations")
    print("=" * 70)
    
    print("""
    Definition:
    ─────────────────────────────────────────────────────────────────────
    
    A matrix is "sparse" if most of its elements are zero.
    
    Example:
    ┌───────────────────────────────────────┐
    │  0  0  0  0  0  0  0  0  0  1        │
    │  0  0  0  0  0  0  0  0  0  0        │
    │  0  0  5  0  0  0  0  0  0  0        │
    │  0  0  0  0  0  0  0  0  0  0        │
    │  0  0  0  0  8  0  0  0  0  0        │
    │  0  0  0  0  0  0  0  0  0  0        │
    │  0  0  0  0  0  0  0  0  0  3        │
    │  0  0  0  0  0  0  0  0  0  0        │
    │  0  0  0  0  0  0  0  0  0  0        │
    │  0  2  0  0  0  0  0  0  0  0        │
    └───────────────────────────────────────┘
    
    10×10 matrix = 100 elements
    Non-zero elements = 5
    Sparsity = 95%
    
    Dense storage: 100 elements
    Sparse storage: 15 values (row, col, value for each non-zero)
    
    ─────────────────────────────────────────────────────────────────────
    
    Why Sparse Matrices Matter:
    ─────────────────────────────────────────────────────────────────────
    
    Applications:
      • Social network graphs (most users don't know most others)
      • Document-term matrices (most words don't appear in most docs)
      • Finite element analysis
      • Image processing
      • Recommendation systems
    
    Memory savings:
      • 1000×1000 matrix with 1% density
      • Dense: 1,000,000 elements
      • Sparse (COO): 10,000 non-zeros × 3 = 30,000 values
      • Savings: 97%!
    """)


sparse_matrix_introduction()
```

---

### **4.6.2 Sparse Matrix Representations**

```python
def sparse_matrix_representations():
    """
    Implement different sparse matrix representations.
    """
    
    print("Sparse Matrix Representations")
    print("=" * 70)
    
    class CoordinateFormat:
        """
        COO (Coordinate) Format - Simplest sparse matrix format.
        
        Stores triplets: (row, column, value)
        
        Good for:
          • Construction (easy to add elements)
          • Conversion to other formats
          • Matrix assembly
        
        Not ideal for:
          • Arithmetic operations
          • Row/column slicing
        """
        
        def __init__(self, rows, cols):
            self.rows = rows
            self.cols = cols
            self.data = []  # List of (row, col, value) tuples
        
        def set(self, row, col, value):
            """Add or update element."""
            if value != 0:
                self.data.append((row, col, value))
        
        def get(self, row, col):
            """Get element value."""
            for r, c, v in self.data:
                if r == row and c == col:
                    return v
            return 0
        
        def to_dense(self):
            """Convert to dense matrix."""
            result = [[0] * self.cols for _ in range(self.rows)]
            for r, c, v in self.data:
                result[r][c] = v
            return result
        
        def nnz(self):
            """Number of non-zero elements."""
            return len(self.data)
        
        def __str__(self):
            return f"COO({self.rows}×{self.cols}, nnz={self.nnz()}): {self.data}"
    
    class CSRFormat:
        """
        CSR (Compressed Sparse Row) Format.
        
        Uses three arrays:
          • values: Non-zero values in row-major order
          • col_indices: Column index for each value
          • row_ptr: Index in values where each row starts
        
        Good for:
          • Row slicing
          • Matrix-vector multiplication
          • Arithmetic operations
        
        Memory: O(nnz + rows + 1)
        """
        
        def __init__(self, rows, cols):
            self.rows = rows
            self.cols = cols
            self.values = []
            self.col_indices = []
            self.row_ptr = [0]  # Row 0 starts at index 0
        
        def from_dense(self, matrix):
            """Build CSR from dense matrix."""
            for i in range(self.rows):
                for j in range(self.cols):
                    if matrix[i][j] != 0:
                        self.values.append(matrix[i][j])
                        self.col_indices.append(j)
                self.row_ptr.append(len(self.values))
            return self
        
        def get(self, row, col):
            """Get element value. O(k) where k is non-zeros in row."""
            start = self.row_ptr[row]
            end = self.row_ptr[row + 1]
            
            for i in range(start, end):
                if self.col_indices[i] == col:
                    return self.values[i]
            return 0
        
        def get_row(self, row):
            """Get entire row efficiently. O(nnz in row)."""
            start = self.row_ptr[row]
            end = self.row_ptr[row + 1]
            
            result = [0] * self.cols
            for i in range(start, end):
                result[self.col_indices[i]] = self.values[i]
            return result
        
        def to_dense(self):
            """Convert to dense matrix."""
            result = []
            for i in range(self.rows):
                result.append(self.get_row(i))
            return result
        
        def nnz(self):
            return len(self.values)
        
        def __str__(self):
            return (f"CSR({self.rows}×{self.cols}, nnz={self.nnz()})\n"
                    f"  values: {self.values}\n"
                    f"  col_indices: {self.col_indices}\n"
                    f"  row_ptr: {self.row_ptr}")
    
    # Demonstration
    dense_matrix = [
        [0, 0, 0, 4, 0],
        [0, 2, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 0, 0, 3, 0],
        [0, 0, 0, 0, 0]
    ]
    
    print("Original Dense Matrix:")
    for row in dense_matrix:
        print(f"  {row}")
    
    print("\n" + "=" * 50)
    print("\nCOO (Coordinate) Format:")
    coo = CoordinateFormat(5, 5)
    for i in range(5):
        for j in range(5):
            if dense_matrix[i][j] != 0:
                coo.set(i, j, dense_matrix[i][j])
    print(coo)
    
    print("\n" + "=" * 50)
    print("\nCSR (Compressed Sparse Row) Format:")
    csr = CSRFormat(5, 5).from_dense(dense_matrix)
    print(csr)
    
    print(f"\nAccessing csr.get(3, 0): {csr.get(3, 0)}")
    print(f"Accessing csr.get(3, 3): {csr.get(3, 3)}")
    print(f"Row 3: {csr.get_row(3)}")
    
    print("""
    
    Comparison of Sparse Formats:
    ─────────────────────────────────────────────────────────────────────
    
    Format   │ Memory          │ Get Element │ Row Access │ Best For
    ─────────┼─────────────────┼─────────────┼────────────┼────────────────
    COO      │ O(3·nnz)        │ O(nnz)      │ O(nnz)     │ Construction
    CSR      │ O(2·nnz + rows) │ O(k) per row│ O(k)       │ Row operations
    CSC      │ O(2·nnz + cols) │ O(k) per col│ O(k)       │ Column operations
    DOK      │ O(3·nnz)        │ O(1)        │ O(cols)    │ Random access
    LIL      │ O(2·nnz)        │ O(k) per row│ O(k)       │ Construction
    
    k = average non-zeros per row/column
    nnz = total non-zero elements
    
    ─────────────────────────────────────────────────────────────────────
    
    Industry Applications:
    
    • SciPy sparse matrices: Multiple formats for different operations
    • Machine Learning: Sparse feature vectors, one-hot encoding
    • Graphs: Adjacency matrices for large sparse graphs
    • NLP: Document-term matrices (TF-IDF)
    """)


sparse_matrix_representations()
```

**Output:**
```
Sparse Matrix Representations
======================================================================
Original Dense Matrix:
  [0, 0, 0, 4, 0]
  [0, 2, 0, 0, 0]
  [0, 0, 0, 0, 0]
  [1, 0, 0, 3, 0]
  [0, 0, 0, 0, 0]

==================================================

COO (Coordinate) Format:
COO(5×5, nnz=4): [(0, 3, 4), (1, 1, 2), (3, 0, 1), (3, 3, 3)]

==================================================

CSR (Compressed Sparse Row) Format:
CSR(5×5, nnz=4)
  values: [4, 2, 1, 3]
  col_indices: [3, 1, 0, 3]
  row_ptr: [0, 1, 2, 2, 4, 4]

Accessing csr.get(3, 0): 1
Accessing csr.get(3, 3): 3
Row 3: [1, 0, 0, 3, 0]


Comparison of Sparse Formats:
─────────────────────────────────────────────────────────────────────

Format   │ Memory          │ Get Element │ Row Access │ Best For
─────────┼─────────────────┼─────────────┼────────────┼────────────────
COO      │ O(3·nnz)        │ O(nnz)      │ O(nnz)     │ Construction
CSR      │ O(2·nnz + rows) │ O(k) per row│ O(k)       │ Row operations
CSC      │ O(2·nnz + cols) │ O(k) per col│ O(k)       │ Column operations
DOK      │ O(3·nnz)        │ O(1)        │ O(cols)    │ Random access
LIL      │ O(2·nnz)        │ O(k) per row│ O(k)       │ Construction

k = average non-zeros per row/column
nnz = total non-zero elements

─────────────────────────────────────────────────────────────────────

Industry Applications:

• SciPy sparse matrices: Multiple formats for different operations
• Machine Learning: Sparse feature vectors, one-hot encoding
• Graphs: Adjacency matrices for large sparse graphs
• NLP: Document-term matrices (TF-IDF)
```

---

## **4.7 Summary and Key Takeaways**

### **4.7.1 Chapter Summary**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    CHAPTER 4 KEY TAKEAWAYS                           │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. STATIC ARRAYS                                                    │
│     • Contiguous memory → O(1) random access                        │
│     • Cache-friendly for sequential access                          │
│     • Insert/delete at position: O(n)                               │
│     • Fixed size determined at compile time                          │
│                                                                      │
│  2. DYNAMIC ARRAYS                                                   │
│     • Automatic resizing when full                                  │
│     • Amortized O(1) append (geometric growth)                      │
│     • Same O(1) random access as static arrays                      │
│     • Industry: Python list, Java ArrayList, C++ vector             │
│                                                                      │
│  3. MULTI-DIMENSIONAL ARRAYS                                        │
│     • Row-major vs column-major order                               │
│     • Cache locality depends on traversal matching layout           │
│     • Index conversion formulas for memory mapping                  │
│                                                                      │
│  4. MATRIX OPERATIONS                                               │
│     • Transpose: O(n²)                                              │
│     • Multiplication: O(n³) standard, O(n^2.81) Strassen            │
│     • Rotation: In-place O(n²) algorithms                           │
│                                                                      │
│  5. SPARSE MATRICES                                                 │
│     • Store only non-zero elements                                  │
│     • COO: Simple, good for construction                           │
│     • CSR: Efficient row operations                                 │
│     • Essential for graphs, NLP, ML                                 │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### **4.7.2 Complexity Quick Reference**

| Operation | Static Array | Dynamic Array |
|-----------|--------------|---------------|
| Access by index | O(1) | O(1) |
| Search (unsorted) | O(n) | O(n) |
| Search (sorted) | O(log n) | O(log n) |
| Insert at end | O(1)* | O(1) amortized |
| Insert at position | O(n) | O(n) |
| Delete at end | O(1) | O(1) amortized |
| Delete at position | O(n) | O(n) |
| Space | Fixed | Variable |

*Only if space is available

---

## **4.8 Practice Problems**

### **Problem 1: Dynamic Array with Shrink**
Modify the DynamicArray implementation to shrink when utilization drops below 25%.

### **Problem 2: Spiral Matrix Traversal**
Given an m×n matrix, return all elements in spiral order (clockwise from outside to inside).

### **Problem 3: Set Matrix Zeroes**
Given an m×n matrix, if an element is 0, set its entire row and column to 0. Do it in O(1) extra space.

### **Problem 4: Sparse Matrix Multiplication**
Implement multiplication of two sparse matrices in CSR format.

### **Problem 5: Rotate Image**
Rotate an n×n image by 90 degrees (clockwise) in-place. Each pixel is 4 bytes.

---

## **4.9 Further Reading**

1. **Introduction to Algorithms (CLRS)** Chapter 10 - Elementary Data Structures
2. **The Art of Computer Programming, Vol 1** by Knuth - Arrays and lists
3. **Programming Pearls** by Jon Bentley - Sparse arrays and space-time trade-offs
4. **SciPy Sparse Matrix Documentation** - Real-world sparse matrix implementations

---

> **Coming in Chapter 5**: We'll explore **Linked Lists**, the complementary data structure to arrays. You'll learn about singly and doubly linked lists, circular lists, cycle detection algorithms (Floyd's algorithm), skip lists for probabilistic data structures, and unrolled linked lists for cache optimization.

---

**End of Chapter 4**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../1. foundations_complexity_analysis/3. programming_fundamentals_for_dsa.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='5. linked_lists.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
